In [116]:
!pip install pyspark

In [117]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,when,count,isnan,isnull,expr

In [118]:
spark=SparkSession.builder.appName('Engine').getOrCreate()

In [119]:
df=spark.read.csv('/content/drive/MyDrive/Flipkart.csv',header=True,inferSchema=True)
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- maincateg: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- actprice1: integer (nullable = true)
 |-- norating1: integer (nullable = true)
 |-- noreviews1: integer (nullable = true)
 |-- star_5f: integer (nullable = true)
 |-- star_4f: integer (nullable = true)
 |-- star_3f: integer (nullable = true)
 |-- star_2f: integer (nullable = true)
 |-- star_1f: integer (nullable = true)
 |-- fulfilled1: integer (nullable = true)



In [120]:
df.show(5)

+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|   id|               title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| 2242|Casuals For Men  ...|   3.8|      Men|Flipkart|      999|    27928|      3543|  14238|   4295|   3457|   1962|   3976|         1|
|20532|Women Black Flats...|   3.9|    Women|Flipkart|      499|     3015|       404|   1458|    657|    397|    182|    321|         1|
|10648|Women Gold Wedges...|   3.9|    Women|Flipkart|      999|      449|        52|    229|     70|     71|     33|     46|         1|
|20677|Men's Height Incr...|   3.9|      Men|Flipkart|     2999|      290|        40|    141|     51|     49|     17|     32|         1|
|12593|Loafers For Men  ...|   3.9|      

In [121]:
df=df.withColumnRenamed("actprice1","Price")
df=df.withColumnRenamed("fulfilled1","Discount")
df=df.withColumnRenamed("title","ProductName")
df.columns

['id',
 'ProductName',
 'Rating',
 'maincateg',
 'platform',
 'Price',
 'norating1',
 'noreviews1',
 'star_5f',
 'star_4f',
 'star_3f',
 'star_2f',
 'star_1f',
 'Discount']

In [122]:
df.describe().show()

+-------+------------------+--------------------+-------------------+---------+--------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+-------------------+
|summary|                id|         ProductName|             Rating|maincateg|platform|             Price|         norating1|        noreviews1|          star_5f|          star_4f|           star_3f|           star_2f|           star_1f|           Discount|
+-------+------------------+--------------------+-------------------+---------+--------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+-------------------+
|  count|              5244|                5244|               5041|     5177|    5244|              5244|              5244|              5244|             5176|             5244|              5244|              5244|    

In [123]:
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+-----------+------+---------+--------+-----+---------+----------+-------+-------+-------+-------+-------+--------+
| id|ProductName|Rating|maincateg|platform|Price|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|Discount|
+---+-----------+------+---------+--------+-----+---------+----------+-------+-------+-------+-------+-------+--------+
|  0|          0|   203|       67|       0|    0|        0|         0|     68|      0|      0|      0|    186|       0|
+---+-----------+------+---------+--------+-----+---------+----------+-------+-------+-------+-------+-------+--------+



In [124]:
clean_df=df.dropna()

In [125]:
fill_df=df.fillna({"Rating":0})

In [126]:
transformed_df = df.withColumn("EffectivePrice", expr("Price - (Price * Discount / 100)"))
transformed_df.select("ProductName", "Price", "Discount", "EffectivePrice").show(5)
high_rated_products = fill_df.filter((col("Rating") > 4) )


high_rated_products.show(5)

+--------------------+-----+--------+--------------+
|         ProductName|Price|Discount|EffectivePrice|
+--------------------+-----+--------+--------------+
|Casuals For Men  ...|  999|       1|        989.01|
|Women Black Flats...|  499|       1|        494.01|
|Women Gold Wedges...|  999|       1|        989.01|
|Men's Height Incr...| 2999|       1|       2969.01|
|Loafers For Men  ...|  999|       0|         999.0|
+--------------------+-----+--------+--------------+
only showing top 5 rows

+-----+--------------------+------+---------+--------+-----+---------+----------+-------+-------+-------+-------+-------+--------+
|   id|         ProductName|Rating|maincateg|platform|Price|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|Discount|
+-----+--------------------+------+---------+--------+-----+---------+----------+-------+-------+-------+-------+-------+--------+
| 6433|ARYA - DIFFERENT ...|   4.2|    Women|Flipkart| 4299|      166|        24|     94|     39|     12|

In [127]:
high_rated_products=transformed_df.filter((col("Rating")>3) & (col("platform")=="Flipkart"))
high_rated_products.show(5)

+-----+--------------------+------+---------+--------+-----+---------+----------+-------+-------+-------+-------+-------+--------+--------------+
|   id|         ProductName|Rating|maincateg|platform|Price|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|Discount|EffectivePrice|
+-----+--------------------+------+---------+--------+-----+---------+----------+-------+-------+-------+-------+-------+--------+--------------+
| 2242|Casuals For Men  ...|   3.8|      Men|Flipkart|  999|    27928|      3543|  14238|   4295|   3457|   1962|   3976|       1|        989.01|
|20532|Women Black Flats...|   3.9|    Women|Flipkart|  499|     3015|       404|   1458|    657|    397|    182|    321|       1|        494.01|
|10648|Women Gold Wedges...|   3.9|    Women|Flipkart|  999|      449|        52|    229|     70|     71|     33|     46|       1|        989.01|
|20677|Men's Height Incr...|   3.9|      Men|Flipkart| 2999|      290|        40|    141|     51|     49|     17|     32|   

In [128]:
high_rated_products=transformed_df.filter((col("EffectivePrice")>3000) & (col("EffectivePrice").isNotNull()) & (col("platform")=="Amazon"))
high_rated_products.show(5)

+-----+--------------------+------+---------+--------+-----+---------+----------+-------+-------+-------+-------+-------+--------+--------------+
|   id|         ProductName|Rating|maincateg|platform|Price|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|Discount|EffectivePrice|
+-----+--------------------+------+---------+--------+-----+---------+----------+-------+-------+-------+-------+-------+--------+--------------+
|18343|Puma Women's Sued...|   4.2|    Women|  Amazon| 4409|       94|        47|     10|      5|      7|     15|     63|       1|       4364.91|
|15945|Skechers Women's ...|   4.3|    Women|  Amazon| 6499|      102|        37|      7|      5|      3|     17|     67|       1|       6434.01|
|16818|Red Chief Men's S...|  NULL|     NULL|  Amazon| 4499|      750|       479|   NULL|      6|     10|     25|     47|       1|       4454.01|
|19567|Red Chief Men's B...|   3.9|      Men|  Amazon| 3799|      847|       541|     12|      6|     11|     22|     49|   

In [129]:
avg_rating=transformed_df.groupBy("platform").agg({"Rating":"avg"})
avg_rating.show()

+--------+------------------+
|platform|       avg(Rating)|
+--------+------------------+
|Flipkart| 4.013193039255362|
|  Amazon|3.9060606060606067|
+--------+------------------+



In [130]:
avg_rating_effectiveprice=transformed_df.groupBy("platform").agg({"EffectivePrice":"avg"})
avg_rating_effectiveprice.show()

+--------+-------------------+
|platform|avg(EffectivePrice)|
+--------+-------------------+
|Flipkart| 1376.5771809337703|
|  Amazon|  986.7551923076921|
+--------+-------------------+



In [131]:
avg_rating_star_5f=transformed_df.groupBy("platform").agg({"star_5f":"count"})
avg_rating_star_5f.show()

+--------+--------------+
|platform|count(star_5f)|
+--------+--------------+
|Flipkart|          5073|
|  Amazon|           103|
+--------+--------------+



In [132]:
transformed_df_gender_notnull=transformed_df.filter(col("maincateg").isNotNull())
avg_rating_bygender=transformed_df_gender_notnull.groupBy("maincateg").agg({"Rating":"avg"})
avg_rating_bygender.show()

+---------+------------------+
|maincateg|       avg(Rating)|
+---------+------------------+
|      Men|3.9662970106075246|
|    Women|4.0416582064297915|
+---------+------------------+



In [133]:
avg_rating_bygender=transformed_df_gender_notnull.groupBy("maincateg").agg({"Rating":"sum"})
avg_rating_bygender.show()

+---------+------------------+
|maincateg|       sum(Rating)|
+---------+------------------+
|      Men| 8226.100000000006|
|    Women|11943.100000000035|
+---------+------------------+



In [134]:
output_display='Flipkart_Data_Insights'
transformed_df.write.mode('overwrite').saveAsTable(output_display)

In [136]:
df1=spark.sql("select * from Flipkart_Data_Insights")
df1.show()

+-----+--------------------+------+---------+--------+-----+---------+----------+-------+-------+-------+-------+-------+--------+--------------+
|   id|         ProductName|Rating|maincateg|platform|Price|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|Discount|EffectivePrice|
+-----+--------------------+------+---------+--------+-----+---------+----------+-------+-------+-------+-------+-------+--------+--------------+
| 2242|Casuals For Men  ...|   3.8|      Men|Flipkart|  999|    27928|      3543|  14238|   4295|   3457|   1962|   3976|       1|        989.01|
|20532|Women Black Flats...|   3.9|    Women|Flipkart|  499|     3015|       404|   1458|    657|    397|    182|    321|       1|        494.01|
|10648|Women Gold Wedges...|   3.9|    Women|Flipkart|  999|      449|        52|    229|     70|     71|     33|     46|       1|        989.01|
|20677|Men's Height Incr...|   3.9|      Men|Flipkart| 2999|      290|        40|    141|     51|     49|     17|     32|   